* Aluno: **Maxuell Lucas de Queiroz de Abreu Cabral**.

* Matrícula: **20210037870**.

# *Trabalho de PCA*

---

**Introdução: Um experimento utilizando PCA na Detecção de Diabetes**

O trabalho realizado envolve a aplicação da Análise de Componentes Principais (PCA) para redução de dimensionalidade em um conjunto de dados relacionado ao diabetes da UCI Machine Learning Repository. Em seguida, você compara o desempenho de modelos de classificação (Regressão Logística e MLP - Multi-Layer Perceptron) treinados nos dados originais e nos dados reduzidos por PCA.

A sequência geral de passos inclui:

Escolha do Conjunto de Dados:

Seleção de um conjunto de dados relacionado ao diabetes do UCI ML Repository.
Pré-processamento:

Carregamento do conjunto de dados e pré-processamento, incluindo normalização dos atributos.
Divisão do Conjunto de Dados:

Divisão do conjunto de dados em conjuntos de treinamento e teste.
Aplicação do PCA:

Utilização da Análise de Componentes Principais (PCA) para reduzir a dimensionalidade do conjunto de treinamento.
Treinamento dos Modelos:

Treinamento de modelos de classificação (Regressão Logística e MLP) nos conjuntos de treinamento originais e reduzidos por PCA.
Avaliação do Desempenho:

Avaliação do desempenho dos modelos nos conjuntos de teste original e reduzido por PCA.
Comparação de Resultados:

Comparação dos resultados, incluindo métricas de desempenho (por exemplo, precisão, recall) e matrizes de confusão.
Análise do Tempo Computacional:

Análise do tempo computacional para treinamento e avaliação dos modelos.
O objetivo é entender como a redução de dimensionalidade afeta o desempenho dos modelos, tanto em termos de qualidade quanto de eficiência computacional. Este tipo de abordagem é útil em situações em que a quantidade de atributos é grande, e a PCA pode ajudar a simplificar o modelo mantendo as informações mais importantes.

 ---

 Aqui abaixo estão as sendo importadas as bibliotecas essenciais para manipulação de dados (pandas e numpy), visualização (matplotlib). As classes e funções específicas são importadas para criação de modelos de redes neurais (MLPClassifier, MLPRegressor), divisão de conjuntos de treinamento e teste (train_test_split), avaliação de modelos (mean_squared_error, accuracy_score, classification_report, confusion_matrix, r2_score), regressão logística (LogisticRegression) e otimização de hiperparâmetros (RandomizedSearchCV), e a principal a do (PCA).

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

Aqui abaixo o código carrega os dados do arquivo 'diabetes.csv' no Google Drive para um DataFrame no Colab e fornece uma visão inicial dos dados, incluindo suas dimensões e estatísticas descritivas.

In [43]:
from google.colab import drive

drive.mount('/content/drive')

import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/diabetes.csv')
df.head()
print(df.shape)
df.describe().transpose()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(768, 9)


,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


Aqui abaixo é selecionado uma coluna alvo (Outcome)  e normaliza as colunas de preditores, dividindo cada valor pelo máximo na respectiva coluna, e em seguida, exibe estatísticas descritivas atualizadas para o DataFrame. Essa normalização é frequentemente realizada para garantir que todas as características tenham uma escala comparável

In [44]:
target_column = ['Outcome']
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,0.226180,0.198210,0.000000,0.058824,0.176471,0.352941,1.0
Glucose,768.0,0.607510,0.160666,0.000000,0.497487,0.587940,0.704774,1.0
BloodPressure,768.0,0.566438,0.158654,0.000000,0.508197,0.590164,0.655738,1.0
SkinThickness,768.0,0.207439,0.161134,0.000000,0.000000,0.232323,0.323232,1.0
Insulin,768.0,0.094326,0.136222,0.000000,0.000000,0.036052,0.150414,1.0
BMI,768.0,0.476790,0.117499,0.000000,0.406855,0.476900,0.545455,1.0
DiabetesPedigreeFunction,768.0,0.194990,0.136913,0.032231,0.100723,0.153926,0.258781,1.0
Age,768.0,0.410381,0.145188,0.259259,0.296296,0.358025,0.506173,1.0
Outcome,768.0,0.348958,0.476951,0.000000,0.000000,0.000000,1.000000,1.0


Nessa parte do código, estamos executando a:

1. **Separação em Atributos e Rótulos:**
   São criadas duas variáveis, `X` e `y`, para representar a matriz de atributos (features) e o vetor de rótulos (labels), respectivamente. Isso divide os dados em informações independentes (atributos) e as variáveis dependentes (rótulos) que o modelo tentará prever.

2. **Divisão em Conjunto de Treinamento e Teste:**
   Utilizando a função `train_test_split` do scikit-learn, o conjunto de dados é dividido em conjuntos de treinamento (`X_train` e `y_train`) e teste (`X_test` e `y_test`). Essa divisão é crucial para treinar o modelo em uma parte dos dados e avaliá-lo em outra, verificando assim seu desempenho em dados não vistos.

3. **Aplicação do PCA:**
   O algoritmo de Análise de Componentes Principais (PCA) é aplicado aos conjuntos de treinamento e teste. O PCA é usado para reduzir a dimensionalidade dos dados, preservando a maior parte da informação. No código, especifica-se o número desejado de componentes principais (`num_components`) e realiza a transformação dos dados usando `fit_transform` para o conjunto de treinamento e `transform` para o conjunto de teste.

4. **Treinamento da Regressão Logística com PCA:**
   Um modelo de Regressão Logística é treinado utilizando os dados resultantes da aplicação do PCA no conjunto de treinamento (`X_train_pca`). Isso permite avaliar como a redução de dimensionalidade afeta o desempenho do modelo de regressão logística.

Essas etapas são fundamentais para a preparação dos dados, a redução de dimensionalidade e o treinamento do modelo, contribuindo para uma abordagem mais eficaz na análise de desempenho de modelos de aprendizado de máquina.

In [45]:

# Separação em atributos e rótulos
X = df[predictors].values
y = df[target_column].values

# Divisão em conjunto de treinamento e teste
test_size = 0.30
develop_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=40)

# Aplicar PCA
num_components = 5  # Defina o número desejado de componentes principais
pca = PCA(n_components=num_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Treinar regressão logística com PCA
lrc_pca = LogisticRegression(random_state=0, max_iter=1000, verbose=True)
lrc_pca.fit(X_train_pca, y_train.ravel())

LogisticRegression(max_iter=1000, random_state=0, verbose=True)

Aqui abaixo é feito a busca de hiperparâmetros para otimizar um modelo de MLP (Multi-Layer Perceptron) aplicado após a redução de dimensionalidade com PCA (Análise de Componentes Principais). O processo é detalhado da seguinte forma:

1. **Definição do Modelo MLP com PCA:**
   É criada uma instância do modelo `MLPClassifier`, que é uma rede neural artificial de múltiplas camadas. O parâmetro `max_iter` é configurado para 2000, determinando o número máximo de iterações durante o treinamento.

2. **Definição do Espaço de Hiperparâmetros:**
   É especificado um conjunto de hiperparâmetros a serem otimizados durante a busca. Isso inclui diferentes arquiteturas de camadas ocultas (`hidden_layer_sizes`), funções de ativação (`activation`), algoritmos de otimização (`solver`), e taxas de aprendizado (`learning_rate`).

3. **Busca Aleatória de Hiperparâmetros com Validação Cruzada (RandomizedSearchCV):**
   Utilizando a função `RandomizedSearchCV` do scikit-learn, é realizada uma busca aleatória dentro do espaço de hiperparâmetros definido. A busca é conduzida com validação cruzada (CV), onde o conjunto de treinamento é dividido em subconjuntos para treinamento e validação.

4. **Treinamento do Modelo MLP com PCA:**
   O modelo MLP é treinado usando os dados de treinamento após a aplicação do PCA (`X_train_pca`). A busca aleatória otimiza os hiperparâmetros para encontrar a combinação que maximiza o desempenho do modelo.

Em resumo, essa seção visa encontrar a configuração ideal de hiperparâmetros para o modelo MLP quando aplicado aos dados reduzidos por PCA. Isso é feito por meio de uma busca aleatória que explora diferentes combinações de parâmetros, visando melhorar o desempenho do modelo na tarefa de classificação.

In [46]:
# Treinar MLP com PCA
mlp_pca = MLPClassifier(max_iter=2000)
parameter_space = {
    'hidden_layer_sizes': [(5, 5, 5), (4, 4, 4), (8, 8, 8)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['sgd', 'adam'],
    'learning_rate': ['constant', 'adaptive']
}
random_search_pca = RandomizedSearchCV(mlp_pca, parameter_space, n_iter=12, cv=5)
random_search_pca.fit(X_train_pca, y_train.ravel())

RandomizedSearchCV(cv=5, estimator=MLPClassifier(max_iter=2000), n_iter=12,
                   param_distributions={'activation': ['tanh', 'relu',
                                                       'logistic'],
                                        'hidden_layer_sizes': [(5, 5, 5),
                                                               (4, 4, 4),
                                                               (8, 8, 8)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'solver': ['sgd', 'adam']})

Treinamento e Avaliação sem PCA

In [47]:
lrc = LogisticRegression(random_state=0, max_iter=1000, verbose=True)
lrc.fit(X_train, y_train.ravel())
regpred_test = lrc.predict(X_test)


Medição do tempo sem PCA

In [48]:
import time
start_time = time.time()
lrc.fit(X_train, y_train.ravel())
training_time_without_pca = time.time() - start_time


Treinamento e Avaliação com PCA

In [49]:
lrc_pca = LogisticRegression(random_state=0, max_iter=1000, verbose=True)
lrc_pca.fit(X_train_pca, y_train.ravel())
regpred_test_pca = lrc_pca.predict(X_test_pca)


Medição do tempo com PCA

In [50]:
start_time = time.time()
lrc_pca.fit(X_train_pca, y_train.ravel())
training_time_with_pca = time.time() - start_time


Comparação de Resultados

In [51]:
print("MATRIZ DE CONFUSAO DO TESTE DA REGRESSAO LOGISTICA SEM PCA")
print(confusion_matrix(y_test, regpred_test))
print(classification_report(y_test, regpred_test))

print("MATRIZ DE CONFUSAO DO TESTE DA REGRESSAO LOGISTICA COM PCA")
print(confusion_matrix(y_test, regpred_test_pca))
print(classification_report(y_test, regpred_test_pca))


MATRIZ DE CONFUSAO DO TESTE DA REGRESSAO LOGISTICA SEM PCA
[[129  13]
 [ 46  43]]
              precision    recall  f1-score   support

           0       0.74      0.91      0.81       142
           1       0.77      0.48      0.59        89

    accuracy                           0.74       231
   macro avg       0.75      0.70      0.70       231
weighted avg       0.75      0.74      0.73       231

MATRIZ DE CONFUSAO DO TESTE DA REGRESSAO LOGISTICA COM PCA
[[128  14]
 [ 48  41]]
              precision    recall  f1-score   support

           0       0.73      0.90      0.81       142
           1       0.75      0.46      0.57        89

    accuracy                           0.73       231
   macro avg       0.74      0.68      0.69       231
weighted avg       0.73      0.73      0.71       231



In [52]:

# Avaliação com MLP
predict_testdev_pca = random_search_pca.predict(pca.transform(X_test))
testscore_pca = accuracy_score(y_test, predict_testdev_pca)
print("test_score dev model com PCA", testscore_pca)
print("MATRIZ DE CONFUSAO DO TESTE DO MODELO COM PCA")
print(confusion_matrix(y_test, predict_testdev_pca))
print(classification_report(y_test, predict_testdev_pca, zero_division=1))

test_score dev model com PCA 0.7402597402597403
MATRIZ DE CONFUSAO DO TESTE DO MODELO COM PCA
[[118  24]
 [ 36  53]]
              precision    recall  f1-score   support

           0       0.77      0.83      0.80       142
           1       0.69      0.60      0.64        89

    accuracy                           0.74       231
   macro avg       0.73      0.71      0.72       231
weighted avg       0.74      0.74      0.74       231



Diferença de Tempo sem PCA e com PCA

In [53]:
print("Diferença de tempo de treinamento:", training_time_without_pca - training_time_with_pca)


Diferença de tempo de treinamento: 0.002002239227294922


**Considerações Finais sobre o Trabalho e o Uso do PCA:**

A incorporação da Análise de Componentes Principais (PCA) no processo de experimentação com modelos de aprendizado de máquina trouxe consigo diversas implicações. Vamos analisar as vantagens e desvantagens observadas ao longo do trabalho:

**Vantagens do PCA:**

1. **Redução de Dimensionalidade:**
   O PCA mostrou-se eficaz na redução da dimensionalidade do conjunto de dados, preservando as principais informações contidas nos atributos. Isso pode ser crucial em conjuntos de dados com muitas variáveis, ajudando a simplificar a modelagem.

2. **Eficiência Computacional:**
   Em alguns casos, a redução de dimensionalidade pode resultar em ganhos de eficiência computacional, como evidenciado pela possível diminuição no tempo de treinamento dos modelos.

3. **Eliminação de Correlações Redundantes:**
   O PCA, ao transformar as variáveis originais em um novo conjunto de variáveis não correlacionadas (componentes principais), ajuda a eliminar redundâncias e multicolinearidade, contribuindo para a estabilidade dos modelos.

**Desvantagens do PCA:**

1. **Interpretabilidade Reduzida:**
   A interpretabilidade dos componentes principais pode ser desafiadora, pois eles são combinações lineares das variáveis originais. Isso pode dificultar a atribuição de significado direto aos componentes.

2. **Perda de Informação:**
   Embora o PCA preserve a variabilidade global dos dados, há uma inevitável perda de informações detalhadas. O número de componentes principais escolhidos influenciará diretamente a quantidade de informação retida.

3. **Sensibilidade a Outliers:**
   O PCA é sensível a outliers, pois busca maximizar a variância total. Outliers podem influenciar significativamente a identificação dos componentes principais.


A aplicação do PCA é uma decisão que deve ser cuidadosamente ponderada em função das características específicas do conjunto de dados e dos objetivos do modelo. Em cenários onde a interpretabilidade dos componentes não é crucial e a eficiência computacional é uma prioridade, o PCA pode ser uma ferramenta valiosa. No entanto, a perda de informação e a necessidade de interpretabilidade devem ser ponderadas em contextos onde esses fatores são críticos.

Ao concluir este trabalho, fica claro que o PCA é uma técnica poderosa, mas sua aplicação deve ser guiada por uma compreensão profunda do problema em questão e das implicações associadas ao uso dessa técnica de redução de dimensionalidade. A escolha entre utilizar ou não o PCA dependerá das necessidades específicas de cada projeto e das características dos dados envolvidos.